In [1]:
import tensorflow as tf
from __future__ import print_function
import numpy as np

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def loadBinData():
# import binary NOTMIST data set
    with np.load("notMNIST.npz") as data :
        Data, Target = data ["images"], data["labels"]
        posClass = 2
        negClass = 9
        dataIndx = (Target==posClass) + (Target==negClass)
        Data = Data[dataIndx]/255.
        Target = Target[dataIndx].reshape(-1, 1)
        Target[Target==posClass] = 1
        Target[Target==negClass] = 0
        np.random.seed(521)
        randIndx = np.arange(len(Data))
        np.random.shuffle(randIndx)
        Data, Target = Data[randIndx], Target[randIndx]
        trainData, trainTarget = Data[:3500], Target[:3500]
        validData, validTarget = Data[3500:3600], Target[3500:3600]
        testData, testTarget = Data[3600:], Target[3600:]
        
    print("Data binary class Loaded")
    print("-------------------------------")
    return trainData, trainTarget,validData, validTarget,\
        testData, testTarget


In [3]:
class BatchSampler(object):
    '''
    A (very) simple wrapper to randomly sample batches without replacement.
    '''
    
    def __init__(self, data, targets, batch_size):
        self.num_points = data.shape[0]
        self.features = data.shape[1]
        self.batch_size = batch_size

        self.data = data
        self.targets = targets

        self.indices = np.arange(self.num_points)

    def random_batch_indices(self, m=None):
        '''
        Get random batch indices without replacement from the dataset.
        If m is given the batch will be of size m.
        Otherwise will default to the class initialized value.
        '''
        if m is None:
            indices = np.random.choice(self.indices, self.batch_size, replace=False)
        else:
            indices = np.random.choice(self.indices, m, replace=False)
        return indices 

    def get_batch(self, m=None):
        '''
        Get a random batch without replacement from the dataset.
        If m is given the batch will be of size m. 
        Otherwise will default to the class initialized value.
        '''
        indices = self.random_batch_indices(m)
        X_batch = np.take(self.data, indices, 0)
        y_batch = self.targets[indices]
        return X_batch, y_batch  


In [4]:
def linearGrad(_data,_target, _weight, _regLambda ):
    '''
    TODO: the actual gradient calculation
    No divide by 2 here because of the 1/2N
    '''
    _target = tf.cast(_target, dtype = tf.float32)
    _data = tf.cast(_data, dtype = tf.float32)
    wtx_min_y = tf.matmul(_data, _weight) - _target
    
    grad = tf.reduce_mean(wtx_min_y) + _regLambda* _weight
    
    return grad

def linearMSE(y_hat, target):
    '''
    TODO: the MSE calculation
    '''
    print("y_hat",  y_hat)
    target = tf.cast(target, dtype = tf.float32)
    print("target", target)
    mse_mat = tf.square(tf.subtract(y_hat, target))
    print("msemst", mse_mat)
    loss = tf.reduce_mean(mse_mat)/2.0
    return loss


def buildLinearGrpah():
    
    W = tf.Variable(tf.truncated_normal(shape=[784, 1], stddev=0.1))
    W = tf.cast(W, dtype=tf.float32)
    #initialize to a gaussian distr, honestly anything would work
    b = tf.Variable(0.0, name='biases')
    

def linearRegression(_data, _target, question):
    '''
    Stochastic gradient descent Q1
    '''
    #shape 28 * 28 = 784
    dataDim1, dum1, dum2 = _data.get_shape().as_list()
    print(dataDim1)
    dataFlat = tf.reshape(_data, [ dataDim1 ,784 ])
    print(dataFlat)
    learningRate = 0.01
 

    W, b = buildLinearGrpah()
    #grad = linearGrad(dataBatch, targetBatch, W, regLambda)
    #gradient calculation
    
    
    '''
    #Q1.1. Tuning the learning rate
    batch size = 500
    7 epoches for training set
    lambda = 0
    num_iter = 20000
    learning_rate = {0.005,0.001,0.0001}  
    plot loss vs number of epoches (and learning rate)
    '''
    if question == 1:
        batch_size = 500
        num_epoch = (dataDim1/batch_size)
        regLambda = 0
        num_iter = 20
        learning_rate = [0.005,0.001,0.0001]
        batch_sampler = BatchSampler(dataFlat.eval(), _target.eval(), batch_size)

    
    currPoch = 0
    for i in range(len(learning_rate)):
        while currPoch < num_epoch:
            #TODO: add num iter loop
            for j in range(0, num_iter):
                dataBatch, targetBatch = batch_sampler.get_batch()
                dataBatch = tf.convert_to_tensor(dataBatch)
                targetBatch = tf.convert_to_tensor(targetBatch)
                currGrad = sess.run(grad, feed_dict:{dataBatch, targetBatch, W, regLambda})
                W = W - grad
                if not (j%50):
                    print(j, " iter", W)
            
            currPoch +=1
            
    print("back from the for loop weight", W)
    print("back from the for loop dataFlat", dataFlat)
    predict_target = tf.matmul(tf.transpose(W), tf.transpose(dataFlat))
    print("goto mse")
    loss = linearMSE(predict_target, _target)
    print("final loss", loss.eval())

 

In [5]:
def linearNormalEqn(_data, _target):
    w_star = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(tf.transpose(_data), _data)), tf.transpose(_data)), _target)
    onesX = tf.ones(shape=tf.pack([tf.shape(_data)[0], 1]))
    appendOnesX = tf.concat(1, [_data, onesX]) #1 for offset b
    pred_y = tf.matmul(appendOnesX, w_star)
    y_hat = tf.cast(tf.greater_equal(pred_y, 0.5), tf.float32) #float to bool to float 1 or 0
    accuracy = tf.reduce_mean(tf.cast(tf.equal(y_hat, _target), tf.float32))
    
    print("######################")
    print("linear normal equation")
    return accuracy

In [6]:
if __name__ == '__main__':
    init = tf.global_variables_initializer()
    sessMain = tf.InteractiveSession()
    sessMain.run(init)

    
    trainData, trainTarget,validData, validTarget,testData, testTarget = loadBinData()
    #data is (3500, 28, 28)
    #the label [1] or [0] is stored in target

    trainData = tf.stack(trainData)
    trainTarget = tf.stack(trainTarget)
    testData = tf.stack(testData)
    testTarget = tf.stack(testTarget)
    validData = tf.stack(validData)
    validtarget = tf.stack(validTarget)
    
    init = tf.global_variables_initializer()
    sess = tf.InteractiveSession()
    sess.run(init)
    
    print("##### Question 1#######")
    trainData  = tf.cast(trainData, dtype = tf.float32)
    linearRegression(trainData, trainTarget, 1)
    print("##########END QUESTION 1#############")

Data binary class Loaded
-------------------------------
##### Question 1#######
3500
Tensor("Reshape:0", shape=(3500, 784), dtype=float32)
0  iter Tensor("sub_1:0", shape=(784, 1), dtype=float32)
0  iter Tensor("sub_41:0", shape=(784, 1), dtype=float32)
0  iter Tensor("sub_81:0", shape=(784, 1), dtype=float32)
0  iter Tensor("sub_121:0", shape=(784, 1), dtype=float32)
0  iter Tensor("sub_161:0", shape=(784, 1), dtype=float32)
0  iter Tensor("sub_201:0", shape=(784, 1), dtype=float32)
0  iter Tensor("sub_241:0", shape=(784, 1), dtype=float32)
back from the for loop weight Tensor("sub_279:0", shape=(784, 1), dtype=float32)
back from the for loop dataFlat Tensor("Reshape:0", shape=(3500, 784), dtype=float32)
goto mse
y_hat Tensor("MatMul_140:0", shape=(1, 3500), dtype=float32)
target Tensor("Cast_282:0", shape=(3500, 1), dtype=float32)
msemst Tensor("Square:0", shape=(3500, 3500), dtype=float32)
final loss nan
##########END QUESTION 1#############


In [7]:
def q1():   
    '''
    #Q1.2 Effect of the mini-batch size 
    batch_size = {500, 1500, 3500}
    learning_rate = result from Q1.1
    lambda = 0
    num_iter = 20000
    TODO: final training MSE for each mini-batch value
    '''
    
    '''
    Q1.3 Generalization
    B = 500, learning rate η = 0.005, and 20000 iterations 
    λ = {0.,0.001,0.1,1}
    find best lambda
    '''
    
    '''
    Q1.4 Comparing SGD with normal equation
    Compare in terms of final training MSE, accuracy and computation time between SGD and normal equation
    Also, discuss when SGD is more practical than using normal equation.
    '''
    
   

In [8]:
def plotFig(_num, x, y, title="default", xLabel="xlabel", yLabel="yLabel", plotLabel ="plotLabel"):
    plt.figure(_num)
    plt.title(title)
    plt.xlabel(xLabel)
    plt.ylabel(yLabel)
    plt.plot(np.array(x), np.array(y), label = plotLabel)
    plt.legend()
    plt.savefig(str(_num) + title + ".png")
    plt.close()
    plt.clf()

In [9]:
def logisticRegression(_data, _target):
    '''
    implement Q2.1 binary logtistic regression
    '''
    #shape 28 * 28 = 784
    dataDim1, dum1, dum2 = _data.get_shape().as_list()
    print(dataDim1)
    dataFlat = tf.reshape(_data, [ dataDim1 ,784 ])
    print(dataFlat)
    learningRate = 0.01
    W = tf.Variable(tf.truncated_normal(shape=[784, 1], stddev=0.1))
    W = tf.cast(W, dtype=tf.float16)
    #initialize to a gaussian distr, honestly anything would work
    b = tf.Variable(0.0, name='biases')
    regLambda = 0
    init = tf.global_variables_initializer()
    sess = tf.InteractiveSession()
    sess.run(init)
    
    
    
    weightedDecayError = tf.reduce_sum( tf.scalar_mul(regLambda, W))
    y_pred = tf.matmul(_data, W)
    crossEntropy = tf.nn.sigmoid_cross_entropy_with_logits(logits = y_predicted, labels = y_target)
    crossEntropyError = tf.reduce_mean(crossEntropy)
    crossEntropyError = tf.add(crossEntropyError, weightedDecayError)
    
    
    
    
    
    '''
    Q2.1.1
    use tf.nn.sigmoid cross entropy with logits
    λ = 0.01 and use 5000 iterations.
    mini-batch size B = 500 
    Plot the best training and validation curves \
        for both cross-entropy loss and classification accuracy vs. the number of epochs 
        \after tuning the learning rate. 
    '''
    
    
    
    
    
    '''
    q2.1.2
    coefficient λ = 0.01
    learning rate η = 0.001
    batch size B = 500
    use tf.train.AdamOptimizer
    Compare the two sets of learning plots
    explain how the Adam optimizer may help learning from the notMNIST dataset. 
    '''
    
    '''
    Q2.1.3
    compare train, validation and test accuracy with normal equation
    zero weight decay and learning rate of 0.001,
        \plot the training cross entropy (or MSE)
        \and accuracy curves for linear( Adam optimize) and logistic regression.
    plot the cross-entropy loss vs squared-error loss 
        \as a function of the prediction yˆ within the interval [0, 1] 
        \and a dummy target y = 0 in 1-D.
    '''
    pass



In [10]:
def loadMultData():
    '''
    load multi-class notMNIST data
    '''
    with np.load("notMNIST.npz") as data:
        Data, Target = data ["images"], data["labels"]
        np.random.seed(521)
        randIndx = np.arange(len(Data))
        np.random.shuffle(randIndx)
        Data = Data[randIndx]/255.
        Target = Target[randIndx]
        trainData, trainTarget = Data[:15000], Target[:15000]
        validData, validTarget = Data[15000:16000], Target[15000:16000]
        testData, testTarget = Data[16000:], Target[16000:]
    return trainData, trainTarget,validData, validTarget, \
        testData, testTarget


In [11]:
def multRegression(_data, _target):
    '''
    question 2.2
    '''
    
    '''
    Q2.2.1 MINIST dataset
    tf.nn.softmax cross entropy with logits
    lambda = 0.01
    batch = 500
    learning_Rate = [tune]
    find best learning rate
    plot the training and validation curves (accuracy vs. the number of epochs.)
    report best test classification accuracy
    '''
    
    '''
    Q2.2.2 faces
    same as 2.2.1 except
    learning_Rate = [tune]
    lambda = [tune]
    batch = 300
    plot the training and validation curves (accuracy vs. the number of epochs.)
    report best test classification accuracy
    report accuracy vs last assignment's KNN
    '''